In [ ]:
import sys
sys.path.append('..')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from mc_coda import estimate_ess
from mc_coda.effective_sample_size import _r_coda
from demo_helper import random_walk_metropolis_sampler, ProductGaussian

## Sample from a multi-variate Gaussian with random-walk Metropolis.

In [ ]:
dim = 100
target_sd = np.arange(1, dim + 1)
gaussian_target = ProductGaussian(marginal_sd=target_sd)
f = lambda x: gaussian_target.compute_logp(x)
proposal_sd = 2.38 / np.sqrt(dim) * target_sd
    # Optimal proposal sd: see Gelman, Roberts, and Gilks 1996.
    
n_samples = 10 ** 4
thin = 5
n_iter = n_samples * thin
x0 = target_sd * np.random.randn(dim)
samples, accept_rate, logp_samples, _ = random_walk_metropolis_sampler(
    f, x0, proposal_sd, n_samples, seed=0, thin=thin
)

## Compare ESS computed by the module (via pure Python codes) with the one computed by R coda package.
\_r\_coda function calls the R coda package; it exists mainly for testing and is not meant for routine uses. The function calls R either via bash or rpy2.

In [ ]:
ar_ess = estimate_ess(samples, axis=-1, method='ar')
r_coda_ess = _r_coda(samples, axis=-1, R_call='external')

In [ ]:
def plot_with_refline(x, y, min_val=None, max_val=None):
    if min_val is None:
        min_val = min(np.min(x), np.min(y))
    if max_val is None:
        max_val = max(np.max(x), np.max(y))
    plt.scatter(
        x, y, s=150, facecolors='none', edgecolors='C0'
    )
    plt.plot([min_val, max_val], [min_val, max_val], color='C1')
    plt.gca().set_aspect('equal', 'box')
    
plt.figure(figsize=(6, 6))
plt.rcParams['font.size'] = 20

plot_with_refline(ar_ess, r_coda_ess)
plt.xlabel('ESS by Python code')
plt.ylabel('ESS by R CODA')
plt.tight_layout()